In [12]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoModel

from transformers import get_cosine_schedule_with_warmup

from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from torch.optim import AdamW

import random

import numpy as np
import pandas as pd

import os

from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
from train import train_epoch, eval_epoch
from model import MegaSiameseModel
from dataset import TopG_Dataset, topg_collate

In [3]:
def single_model(model, 
                     dataset, 
                     loss_function,
                     collate_fn,
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle=True,
                     epochs: int=15, 
                     lr: float=1e-6,
                     batch_size: int=32,
                     start_epoch=0,
                     ):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    model.to(device)
    optimizer = AdamW(
        model.parameters(),
#        lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
    )
    data_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                     collate_fn=collate_fn
    )
    
    total_steps = len(data_loader) * epochs 

    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

    for epoch_i in range(0, epochs):
        if epoch_i >= start_epoch:
            train_metrics = train_epoch(model, data_loader, loss_function, optimizer, scheduler, device)
            print("EPOCH", epoch_i)
            print(train_metrics)
            #eval_metrics = eval_epoch(model, eval_loader, loss_function, device)
            #print(eval_metrics)

In [4]:
DATA = "data/"
train = pd.read_csv(f"{DATA}train.csv")

In [5]:

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

#model = AutoModelForPreTraining.from_pretrained("cointegrated/rubert-tiny2")
bert1 = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
bert2 = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model = MegaSiameseModel(bert1, bert2, 2)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.

In [6]:
dataset = TopG_Dataset(train, tokenizer, 200, 60)

In [15]:
def cross_validation(project_name,
                     model, 
                     dataset, 
                     loss_function,
                     collate_fn,
                     strat_array=None,
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle: bool=True, 
                     n_folds: int=4, 
                     epochs: int=5, 
                     lr: float=1e-6,
                     start_fold: int=0, 
                     batch_size: int=32,
                     iters_to_accumulate=None,
                     n_accumulated_grads: int = 0):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    if strat_array:
        kfold = StratifiedKFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset, strat_array)
    else: 
        kfold = KFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset)

    for fold, (train_ids, eval_ids) in enumerate(split):
        if fold >= start_fold:
            print(f'FOLD {fold}')
            print('--------------------------------')
            
            
            '''run = wandb.init(
                name=f"fold_{fold}",
                project=f"{project_name}_fold_{fold}",
                config={ 
                         "random_state": random_state, 
                         "shuffle": shuffle,
                         "epochs": epochs, 
                         "learning_rate": lr,
                         "batch_size": batch_size,
                         "iters_to_accumulate": iters_to_accumulate
                        }
            )'''

            optimizer = AdamW(
            model.parameters(),
            #lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
        )

            train_subsampler = torch.utils.data.Subset(dataset,  train_ids)
            train_loader = torch.utils.data.DataLoader(
                          train_subsampler, 
                          batch_size=batch_size,
                          shuffle=shuffle,collate_fn=collate_fn,drop_last=True)

            eval_subsampler = torch.utils.data.Subset(dataset,  eval_ids)
            eval_loader = torch.utils.data.DataLoader(
                          eval_subsampler,
                          batch_size=batch_size,
                          shuffle=shuffle,collate_fn=collate_fn,drop_last=True)
            
            total_steps = len(train_loader) * epochs 

            scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)

            mrrs = []

            for epoch_i in range(0, epochs):
                train_metrics = train_epoch(model, train_loader, loss_function, optimizer, scheduler, device)
                eval_metrics = eval_epoch(model, eval_loader, loss_function, device)
                
                print(f"EPOCH: {epoch_i}")
                print(train_metrics)
                print(eval_metrics)
                
                #run.log(train_metrics)
                #run.log(eval_metrics)
                            

In [ ]:
cross_validation("A", model = model,
             dataset=dataset, 
             loss_function=nn.CrossEntropyLoss(), 
             collate_fn=topg_collate,
             device=torch.device("cuda"),
                     random_state=69,
                     shuffle=True)

FOLD 0
--------------------------------


  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

EPOCH: 0
{'Train Loss': 0.31681531455574763, 'Train Accuracy': 0.8834635615348816, 'Train F1*100': 88.03209271227992}
{'Eval Loss': 0.2474297343287617, 'Eval Accuracy': 0.9049479365348816, 'Eval F1': 90.44502617801048}


  0%|          | 0/144 [00:00<?, ?it/s]

...

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

EPOCH: 2
{'Train Loss': 0.24141717875479823, 'Train Accuracy': 0.9203559160232544, 'Train F1*100': 91.85349611542729}
{'Eval Loss': 0.2231445969082415, 'Eval Accuracy': 0.9361979365348816, 'Eval F1': 93.14685314685313}


  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]